# Trend Factor Analysis

This notebook walks through a trend factor analysis, for both time series and cross-sectional strategies, in digital assets.

We will construct various trend factors and compare their performance across a range of statistical tests and financial evaluation criteria.

Trend following is a ubiquitous factor investing strategy and remains one of the best ways to capture convexity in cryptoassets.

To conduct this analysis, we will use **FactorLab**, an open-source python package we have created specifically for alpha and risk factor analysis. 

To install **FactorLab**:
`pip install factorlab`

In [ ]:
# uncomment to install factorlab
# pip install factorlab

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import percentileofscore

from factorlab.feature_engineering.transformations import Transform
from factorlab.feature_engineering.factors.trend import Trend
from factorlab.signal_generation.signal import Signal
from factorlab.strategy_analysis.feature_selection import FeatureSelection
from factorlab.signal_generation.time_series_analysis import rolling_window, expanding_window, TimeSeriesAnalysis as TSA
from factorlab.strategy_analysis.factor_models import FactorModel
from factorlab.strategy_backtesting.portfolio_optimization._portfolio_optimization import PortfolioOptimization
from factorlab.strategy_analysis.portfolio_sort import PortfolioSort
from factorlab.strategy_backtesting.performance import Performance
from factorlab.strategy_backtesting.metrics import Metrics
from factorlab.data_viz.plot import plot_series, plot_bar, plot_table, plot_scatter, plot_heatmap

## Data 

We stitch the following OHLCV prices to create an extended price and funding rate history:

- Funding rates and perptual futures OHLC prices from **Binance** futures exchange (2019-present)
- OHLC spot prices from **Binance** spot exchange (2017-2019)
- OHLC spot prices from **Cryptocompare** (2010-2017)

In [ ]:
df = pd.read_csv('binance_historical_ohlcv_daily.csv', index_col=['date', 'ticker'], parse_dates=['date'])

In [ ]:
print(f'Number of assets: {df.index.get_level_values(1).unique().shape[0]}')

In [ ]:
# ohlc
ohlc = df[['open', 'high', 'low', 'close']]

In [ ]:
ohlc.head()

## Target and Factor Construction

### Target Returns

The **`Transform`** class allows us to easily create target variables with spot price and total returns for our universe of cryptoassets.

In [ ]:
# compute total returns
ret_df = Transform(df.close).returns()
ret_df = pd.concat([ret_df, df.funding_rate], axis=1).dropna()
ret_df['tr'] = ret_df.close.subtract(ret_df.funding_rate, axis=0)

In [ ]:
# compute forward returns
fwd_spot_ret = Transform(df.close).returns(lags=1, forward=True).to_frame('fwd_ret_1')
fwd_spot_ret['fwd_ret_5'] = Transform(df.close).returns(lags=5, forward=True)
fwd_spot_ret['fwd_ret_7'] = Transform(df.close).returns(lags=7, forward=True)
fwd_spot_ret['fwd_ret_10'] = Transform(df.close).returns(lags=10, forward=True)
fwd_spot_ret['fwd_ret_14'] = Transform(df.close).returns(lags=14, forward=True)
fwd_spot_ret['fwd_ret_20'] = Transform(df.close).returns(lags=20, forward=True)
fwd_spot_ret['fwd_ret_30'] = Transform(df.close).returns(lags=30, forward=True)
fwd_spot_ret['fwd_ret_60'] = Transform(df.close).returns(lags=60, forward=True)

In [ ]:
# compute fwd relative returns
fwd_rel_ret = fwd_spot_ret.copy()
for col in fwd_rel_ret.columns:
    fwd_rel_ret[col] = fwd_spot_ret[col] - fwd_spot_ret[col].groupby('date').mean()

We can also use the **`Transform`** class to standardize returns and create a market portfolio return.

In [ ]:
# normalize fwd rets
fwd_ret_norm = Transform(fwd_spot_ret).normalize(window_type='expanding')
fwd_rel_ret_norm = Transform(fwd_rel_ret).normalize(window_type='expanding')

In [ ]:
# mkt returns
mkt_ret = Transform(df).returns(market=True, mkt_field='close')

In [ ]:
mkt_ret.head()

### Trend Factors

The **`Trend`** class allows us to use compute a dozen or so trend factor factors across varying lookback windows. 

In [ ]:
# trend factors
# breakout
trend_df = Trend(ohlc, vwap=True, log=True, lookback=5).breakout()
trend_df['breakout_10'] = Trend(ohlc, vwap=True, log=True, lookback=10).breakout()
trend_df['breakout_15'] = Trend(ohlc, vwap=True, log=True, lookback=15).breakout()
trend_df['breakout_30'] = Trend(ohlc, vwap=True, log=True, lookback=30).breakout()
trend_df['breakout_45'] = Trend(ohlc, vwap=True, log=True, lookback=45).breakout()
trend_df['breakout_60'] = Trend(ohlc, vwap=True, log=True, lookback=60).breakout()
trend_df['breakout_90'] = Trend(ohlc, vwap=True, log=True, lookback=90).breakout()
trend_df['breakout_180'] = Trend(ohlc, vwap=True, log=True, lookback=180).breakout()
trend_df['breakout_365'] = Trend(ohlc, vwap=True, log=True, lookback=365).breakout()
# price mom
trend_df['price_mom_5'] = Trend(ohlc, vwap=True, log=True, lookback=5).price_mom()
trend_df['price_mom_10'] = Trend(ohlc, vwap=True, log=True, lookback=10).price_mom()
trend_df['price_mom_15'] = Trend(ohlc, vwap=True, log=True, lookback=15).price_mom()
trend_df['price_mom_30'] = Trend(ohlc, vwap=True, log=True, lookback=30).price_mom()
trend_df['price_mom_45'] = Trend(ohlc, vwap=True, log=True, lookback=45).price_mom()
trend_df['price_mom_60'] = Trend(ohlc, vwap=True, log=True, lookback=60).price_mom()
trend_df['price_mom_90'] = Trend(ohlc, vwap=True, log=True, lookback=90).price_mom()
trend_df['price_mom_180'] = Trend(ohlc, vwap=True, log=True, lookback=180).price_mom()
trend_df['price_mom_365'] = Trend(ohlc, vwap=True, log=True, lookback=365).price_mom()
# moving window diff
trend_df['mw_diff_5'] = Trend(ohlc, vwap=True, log=True, lookback=5, sm_window_type='ewm').mw_diff()
trend_df['mw_diff_10'] = Trend(ohlc, vwap=True, log=True, lookback=10, sm_window_type='ewm').mw_diff()
trend_df['mw_diff_15'] = Trend(ohlc, vwap=True, log=True, lookback=15, sm_window_type='ewm').mw_diff()
trend_df['mw_diff_30'] = Trend(ohlc, vwap=True, log=True, lookback=30, sm_window_type='ewm').mw_diff()
trend_df['mw_diff_45'] = Trend(ohlc, vwap=True, log=True, lookback=45, sm_window_type='ewm').mw_diff()
trend_df['mw_diff_60'] = Trend(ohlc, vwap=True, log=True, lookback=60, sm_window_type='ewm').mw_diff()
trend_df['mw_diff_90'] = Trend(ohlc, vwap=True, log=True, lookback=90, sm_window_type='ewm').mw_diff()
trend_df['mw_diff_180'] = Trend(ohlc, vwap=True, log=True, lookback=180, sm_window_type='ewm').mw_diff()
trend_df['mw_diff_365'] = Trend(ohlc, vwap=True, log=True, lookback=365, sm_window_type='ewm').mw_diff()
# divergence
trend_df['divergence_5'] = Trend(ohlc, vwap=True, log=True, lookback=5, sm_window_type='ewm').divergence()
trend_df['divergence_10'] = Trend(ohlc, vwap=True, log=True, lookback=10, sm_window_type='ewm').divergence()
trend_df['divergence_15'] = Trend(ohlc, vwap=True, log=True, lookback=15, sm_window_type='ewm').divergence()
trend_df['divergence_30'] = Trend(ohlc, vwap=True, log=True, lookback=30, sm_window_type='ewm').divergence()
trend_df['divergence_45'] = Trend(ohlc, vwap=True, log=True, lookback=45, sm_window_type='ewm').divergence()
trend_df['divergence_60'] = Trend(ohlc, vwap=True, log=True, lookback=60, sm_window_type='ewm').divergence()
trend_df['divergence_90'] = Trend(ohlc, vwap=True, log=True, lookback=90, sm_window_type='ewm').divergence()
trend_df['divergence_180'] = Trend(ohlc, vwap=True, log=True, lookback=180, sm_window_type='ewm').divergence()
trend_df['divergence_365'] = Trend(ohlc, vwap=True, log=True, lookback=365, sm_window_type='ewm').divergence()
# exp weighted mov avg crossover
trend_df['ewma_xover_2'] = Trend(ohlc, vwap=True, log=True,).ewma_wxover(s_k=[2, 4, 8], l_k=[6, 12, 24], signal=True)
trend_df['ewma_xover_3'] = Trend(ohlc, vwap=True, log=True,).ewma_wxover(s_k=[3, 6, 12], l_k=[9, 18, 36], signal=True)
trend_df['ewma_xover_4'] = Trend(ohlc, vwap=True, log=True,).ewma_wxover(s_k=[4, 8, 16], l_k=[12, 24, 48], signal=True)
trend_df['ewma_xover_5'] = Trend(ohlc, vwap=True, log=True,).ewma_wxover(s_k=[5, 10, 20], l_k=[15, 30, 60], signal=True)
# rsi
trend_df['rsi_5'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=5).rsi()
trend_df['rsi_10'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=10).rsi()
trend_df['rsi_15'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=15).rsi()
trend_df['rsi_30'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=30).rsi()
trend_df['rsi_45'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=45).rsi()
trend_df['rsi_60'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=60).rsi()
trend_df['rsi_90'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=90).rsi()
trend_df['rsi_180'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=180).rsi()
trend_df['rsi_365'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=365).rsi()
# stochastic
trend_df['stoch_5'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=5).stochastic()
trend_df['stoch_10'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=10).stochastic()
trend_df['stoch_15'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=15).stochastic()
trend_df['stoch_30'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=30).stochastic()
trend_df['stoch_45'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=45).stochastic()
trend_df['stoch_60'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=60).stochastic()
trend_df['stoch_90'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=90).stochastic()
trend_df['stoch_180'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=180).stochastic()
trend_df['stoch_365'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=365).stochastic()
# intensity
trend_df['intensity_5'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=5).intensity()
trend_df['intensity_10'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=10).intensity()
trend_df['intensity_15'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=15).intensity()
trend_df['intensity_30'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=30).intensity()
trend_df['intensity_45'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=45).intensity()
trend_df['intensity_60'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=60).intensity()
trend_df['intensity_90'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=90).intensity()
trend_df['intensity_180'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=180).intensity()
trend_df['intensity_365'] = Trend(ohlc, vwap=True, log=True, sm_window_type='ewm', lookback=365).intensity()
# time trend
trend_df['time_trend_5'] = Trend(ohlc, vwap=True, log=True, lookback=5).time_trend()
trend_df['time_trend_10'] = Trend(ohlc, vwap=True, log=True, lookback=10).time_trend()
trend_df['time_trend_15'] = Trend(ohlc, vwap=True, log=True, lookback=15).time_trend()
trend_df['time_trend_30'] = Trend(ohlc, vwap=True, log=True, lookback=30).time_trend()
trend_df['time_trend_45'] = Trend(ohlc, vwap=True, log=True, lookback=45).time_trend()
trend_df['time_trend_60'] = Trend(ohlc, vwap=True, log=True, lookback=60).time_trend()
trend_df['time_trend_90'] = Trend(ohlc, vwap=True, log=True, lookback=90).time_trend()
trend_df['time_trend_180'] = Trend(ohlc, vwap=True, log=True, lookback=180).time_trend()
trend_df['time_trend_365'] = Trend(ohlc, vwap=True, log=True, lookback=365).time_trend()
# alpha mom
trend_df['alpha_mom_5'] = Trend(ohlc, vwap=True, log=True, lookback=5).alpha_mom()
trend_df['alpha_mom_10'] = Trend(ohlc, vwap=True, log=True, lookback=10).alpha_mom()
trend_df['alpha_mom_15'] = Trend(ohlc, vwap=True, log=True, lookback=15).alpha_mom()
trend_df['alpha_mom_30'] = Trend(ohlc, vwap=True, log=True, lookback=30).alpha_mom()
trend_df['alpha_mom_45'] = Trend(ohlc, vwap=True, log=True, lookback=45).alpha_mom()
trend_df['alpha_mom_60'] = Trend(ohlc, vwap=True, log=True, lookback=60).alpha_mom()
trend_df['alpha_mom_90'] = Trend(ohlc, vwap=True, log=True, lookback=90).alpha_mom()
trend_df['alpha_mom_180'] = Trend(ohlc, vwap=True, log=True, lookback=180).alpha_mom()
trend_df['alpha_mom_365'] = Trend(ohlc, vwap=True, log=True, lookback=365).alpha_mom()
# energy
trend_df['energy_5'] = Trend(ohlc, vwap=True, log=True, lookback=5).energy()
trend_df['energy_10'] = Trend(ohlc, vwap=True, log=True, lookback=10).energy()
trend_df['energy_15'] = Trend(ohlc, vwap=True, log=True, lookback=15).energy()
trend_df['energy_30'] = Trend(ohlc, vwap=True, log=True, lookback=30).energy()
trend_df['energy_45'] = Trend(ohlc, vwap=True, log=True, lookback=45).energy()
trend_df['energy_60'] = Trend(ohlc, vwap=True, log=True, lookback=60).energy()
trend_df['energy_90'] = Trend(ohlc, vwap=True, log=True, lookback=90).energy()
trend_df['energy_180'] = Trend(ohlc, vwap=True, log=True, lookback=180).energy()
trend_df['energy_365'] = Trend(ohlc, vwap=True, log=True, lookback=365).energy()

The **`plot_series`** function from the data_viz.plot module allows us to plot the trend factors for Bitcoin:

In [ ]:
plot_series(trend_df.loc[:, 'BTC', :].price_mom_30,
            title='30 Day Price Momentum',
            subtitle='Bitcoin'
           )

# Signal Generation

The **`Signal`** class can be used to convert raw alpha factors to either a time series or cross-sectional trading signals with the **`compute_signals`** method. To initialize the **`Signal`** class, we can use the `returns` and `factors` parameters, and specify the type of strategy with `strategy` to either time series `ts` or cross-sectional `cs` followed by either long/short `ls`, long-only `l` or short-only `s`. 

The `signal_type` argument in the **`compute_signals`** method can be set to either a continuous signal `signal` with continuous values between -1 and 1 (default), a discrete signal `disc_signal` with discrete values -1, 0, 1, signal quantiles `signal_quantiles` with discrete values between -1 and 1 (e.g. -1, -0.5, 0 , 0.5, 1 for signal_quantiles with `factor_bins` = 5) and signal rank `signal_rank` with values of -1, 0 and 1 for long/short cross-sectional strategies with the top/bottom `n_factors`.

In [ ]:
trend_signals_ts_df = Signal(ret_df.tr, trend_df, strategy='ts_ls', disc_thresh=0).compute_signals(signal_type='signal')

We plot the time series signals for Bitcoin for the price momentum `price_mom_30` trend factor with 30 period window lookback:

In [ ]:
plot_series(trend_signals_ts_df.loc[:, 'BTC', :].price_mom_30, 
            title='Trend Factor: Bitcoin - 30 Day Price Momentum',
            subtitle='Time Series Strategy'
           )

In [ ]:
trend_signals_cs_df = Signal(ret_df.tr, trend_df, strategy='cs_ls').compute_signals(signal_type='signal')

We can also plot the cross-sectional signals for Bitcoin, Ethereum and Solana for the price momentum `price_mom_30` trend factor with 30 period window lookback:

In [ ]:
plot_series(trend_signals_cs_df.loc[:, ['BTC', 'ETH', 'SOL'], :].price_mom_30.unstack(), 
            title='Trend Factor: 30 Day Price Momentum',
            subtitle='Cross-sectional Strategy'
           )

## Signals vs. Forward Returns

The **`plot_scatter`** function can be used to plot signals vs. forward returns to visualize the relationship between alpha factor signals and the forward returns.

In [ ]:
# ts trend signal vs. forward returns
price_mom_30_ts_signals_vs_ret = pd.concat([trend_signals_ts_df.price_mom_30, fwd_ret_norm.fwd_ret_7], axis=1).dropna()

In [ ]:
# ts scatter plot
plot_scatter(price_mom_30_ts_signals_vs_ret, x='price_mom_30', y='fwd_ret_7', hue=price_mom_30_ts_signals_vs_ret.fwd_ret_7.abs(), title=f'{trend_signals_ts_df.price_mom_30.name} vs. Forward Returns (Standardized)', subtitle='Time Series Strategy')

In [ ]:
# cs trend signal vs. forward returns
price_mom_30_cs_signals_vs_ret = pd.concat([trend_signals_cs_df.price_mom_30, fwd_rel_ret_norm.fwd_ret_7], axis=1).dropna()

In [ ]:
# cs scatter plot
plot_scatter(price_mom_30_cs_signals_vs_ret, x='price_mom_30', y='fwd_ret_7', hue=price_mom_30_cs_signals_vs_ret.fwd_ret_7.abs(), title=f'{trend_signals_cs_df.price_mom_30.name} vs. Relative Forward Returns (Standardized)', subtitle='Cross-sectional Strategy')

## Statistical Tests

Statistical tests allow us to assess the strenght of the relationship between the alpha factors (features) and foward returns (target variable).


### Feature Selection: 
Correlation measures, e.g. spearman rank correlation (aka information coefficient), and association measures, e.g. mutul information, allow us to evaluate the predictive relationship between an alpha factors and forward returns.

The **`FeatureSelection`** class has a **`filter`** method for both time series `ts` and cross-sectional `cs` strategies which will rank alpha factors on the strength of their relationship with forward returns. The correlation or association measure can be specified by the `method` parameter.


In [ ]:
FeatureSelection(fwd_ret_norm.fwd_ret_7, trend_signals_ts_df, strategy='ts', feature_bins=5, target_bins=3, window_type='fixed').filter(method='spearman_rank')

In [ ]:
FeatureSelection(fwd_rel_ret_norm.fwd_ret_7, trend_signals_cs_df, strategy='cs', feature_bins=5, target_bins=3, window_type='fixed').filter(method='spearman_rank')

### Information Coefficient (IC):
The information coefficient **`ic`** method (aka spearman rank correlation) can be computed over a rolling window to allow us to see the change in predictive relationship between the factors and forward returns.


In [ ]:
# IC rolling window for time series strategy
ic_ts_df = FeatureSelection(fwd_ret_norm.fwd_ret_7, trend_signals_ts_df, strategy='ts', feature_bins=5, target_bins=3, window_size=5).ic(feature='price_mom_30')

In [ ]:
plot_series(ic_ts_df.dropna(), 
            title='30 Day Price Momentum - Information Coefficient',
            subtitle='Time series, 365 day rolling window'
           )

In [ ]:
# IC rolling window for cross sectional strategy
ic_cs_df = FeatureSelection(fwd_rel_ret_norm.fwd_ret_7, trend_signals_cs_df, strategy='cs', feature_bins=5, target_bins=3, window_size=30).ic(feature='price_mom_30')

In [ ]:
plot_series(ic_cs_df.dropna(), 
            title='Information Coefficient',
            subtitle='Cross-sectional, 365 day rolling window'
           )

### Regressions

The **`FactorModel`** class has both a **`pooled_regression`** (time series) and **`fama_macbeth`** method (cross-sectional) which can be used to assess he statistical and economic significance of factors by regressing them on forward returns.

In [ ]:
# pooled regression for time series 
FactorModel(fwd_ret_norm.fwd_ret_7, trend_signals_ts_df, strategy='ts').pooled_regression(multivariate=False)

In [ ]:
# cross sectional fama-macbeth
FactorModel(fwd_rel_ret_norm.fwd_ret_7, trend_signals_cs_df, strategy='cs').fama_macbeth_regression(multivariate=False)

## Signal Returns

The **`Signal`** class has a **`compute_signal_returns`** method which can compute alpha factor returns, signals * next period return. 

In [ ]:
# trend signal ts returns
trend_signals_ts_ret = Signal(ret_df.tr, trend_df, strategy='ts_ls').compute_signal_returns(signal_type='signal')

In [ ]:
# asset specific trend signal
asset = 'BTC'
trend_signal_ts_ret = trend_signals_ts_ret.loc[:, asset, :]

In [ ]:
# ts trend signal performance
trend_signal_ts_perf_table = Performance(trend_signal_ts_ret, mkt_ret=mkt_ret, ret_type='log').get_table(metrics='all', rank_on='Sharpe ratio')

In [ ]:
trend_signal_ts_perf_table.iloc[:20]

In [ ]:
trend_signal_ts_perf_table['Sharpe ratio'].plot(kind='hist');

In [ ]:
# buy & hold asset
hodl_perf = Metrics(ret_df.tr.loc[:, asset, :]).sharpe_ratio().tr
hodl_perf.round(2)

In [ ]:
# percentile rank of the buy and hold Sharpe ratio
percentile = percentileofscore(trend_signal_ts_perf_table['Sharpe ratio'], hodl_perf)
print(f'Hodling is outperformed by {100 - percentile.round(2)}% of trend factor signals in {asset}.')

In [ ]:
# ts trend cum returns 
trend_signal_ts_cum_ret = Metrics(trend_signal_ts_ret, mkt_ret=mkt_ret, ret_type='log', window_type='expanding').cumulative_returns()

In [ ]:
plot_series(trend_signal_ts_cum_ret, 
            y_label='Cumulative returns', 
            title=f'Trend Signal: {asset}',
            subtitle='Time Series Strategy',
           )

# Strategy Portfolio Returns

The **`PortfolioOptimization`** class includes over a dozen portfolio optimizers which allow the computation of optimal asset weights:

In [ ]:
PortfolioOptimization(trend_signals_ts_ret).get_available_optimizers()

## Time Series Strategies

We can create an instance of the **`PortfolioOptimization`** class. Instance attributes include optimization methods `method`, transaction cost `t_cost`, rebalancing frequency `rebal_freq`, window type `window_type` and window size `window_size`.

Below, we use create a instance with equal weights, t-costs of 20 bps, rebalanced weekly over a lookback window size of 180 periods.

In [ ]:
# ts trend signal returns
trend_signals_ts_ret = Signal(ret_df.tr, trend_df, strategy='ts_ls').compute_signal_returns(signal_type='signal')

In [ ]:
# ts trend portfolio optimization instance
port_opt = PortfolioOptimization(trend_signals_ts_ret, 
                                 method='equal_weight', 
                                 t_cost=0.002, 
                                 rebal_freq=7, 
                                 window_type='rolling',
                                 window_size=180)

In [ ]:
# ts trend portfolio returns
trend_ts_port_ret = port_opt.compute_portfolio_returns()

In [ ]:
# ts trend portfolio performance
trend_ts_port_perf_table = Performance(trend_ts_port_ret, mkt_ret=mkt_ret, ret_type='log').get_table(metrics='all', rank_on='Sharpe ratio')

In [ ]:
trend_ts_port_perf_table.iloc[:20]

In [ ]:
# ts trend cum returns 
trend_ts_cum_ret = Metrics(trend_ts_port_ret.fillna(0), mkt_ret=mkt_ret, ret_type='log', window_type='expanding').cumulative_returns()

In [ ]:
plot_series(trend_ts_cum_ret,             
            y_label='Cumulative returns', 
            title='Trend Factor: Portfolio Returns',
            subtitle='Time Series')

In [ ]:
trend_ts_port_perf_table.mean()

## Cross-Sectional Strategies

Cross-sectional strategies sorts assets into portfolios based on their cross-sectional factor values (i.e. relative factor values at each timestamp). Signals for each portfolio can be continuous (from 1 to -1) `signal`, discrete (1, 0, -1) `disc_signal` for the top and bottom quantiles, signal quantiles (e.g. 1, 0.5, 0, -0.5, -1) `signal_quantiles` scaled on factor quantiles, or signal rank `signal_rank` which ranks factor values by top/bottom n assets.

In [ ]:
# trend signal cs returns
trend_signals_cs_ret = Signal(ret_df.tr, trend_df, strategy='cs_ls').compute_signal_returns(signal_type='signal')

In [ ]:
# cs trend portfolio returns
trend_cs_port_ret = PortfolioOptimization(trend_signals_cs_ret, 
                                          method='equal_weight', 
                                          t_cost=0.002, 
                                          rebal_freq=7, 
                                          window_type='rolling',
                                          window_size=180).compute_portfolio_returns()

In [ ]:
# cs trend portfolio performance
trend_cs_port_perf_table = Performance(trend_cs_port_ret, mkt_ret=mkt_ret, ret_type='log').get_table(metrics='all', rank_on='Sharpe ratio')

In [ ]:
trend_cs_port_perf_table

In [ ]:
# cs trend cum returns
trend_cs_cum_ret = Metrics(trend_cs_port_ret.fillna(0), mkt_ret=mkt_ret, ret_type='log', window_type='expanding').cumulative_returns()

In [ ]:
plot_series(trend_cs_cum_ret.dropna(),            
            y_label='Cumulative returns', 
            title='Trend Factor: Portfolio Returns',
            subtitle='Cross-sectional')

In [ ]:
trend_cs_port_perf_table.mean()

## Dual Strategies
Dual strategies (e.g. dual momentum) go long/short assets with factors that are high/low in both the cross section (relative to other assets) and in the time series (relative to the asset's own history). Dual strategies can enhance risk-adjusted returns by combining both time series and cross sectional strategies.

In [ ]:
# trend signal dual returns
trend_signals_dual_ret = Signal(ret_df.tr, trend_df, strategy='dual_ls').compute_signal_returns(signal_type='signal', dual_summary_stat='sum')

In [ ]:
# trend dual portfolio returns
trend_dual_port_ret = PortfolioOptimization(trend_signals_dual_ret, 
                                            method='equal_weight', 
                                            t_cost=0.002, 
                                            rebal_freq=7, 
                                            window_type='rolling',
                                            window_size=180).compute_portfolio_returns()

In [ ]:
# dual trend portfolio performance
trend_dual_port_perf_table = Performance(trend_dual_port_ret, mkt_ret=mkt_ret, ret_type='log').get_table(metrics='all', rank_on='Sharpe ratio')

In [ ]:
trend_dual_port_perf_table[:20]

In [ ]:
# dual trend cum returns
trend_dual_cum_ret = Metrics(trend_dual_port_ret, mkt_ret=mkt_ret, ret_type='log', window_type='expanding').cumulative_returns()

In [ ]:
plot_series(trend_dual_cum_ret.dropna(),            
            y_label='Cumulative returns', 
            title='Trend Factor: Portfolio Returns',
            subtitle='Dual Momentum')

In [ ]:
trend_dual_port_perf_table.mean()

# Portfolio Sorts

The **`PortfolioSort`** class allows us to sort returns into quantile portfolio returns which provide a measure of alpha factor performance. The **`compute_quantile_portfolios`** method compute quantile portfolio returns, and the **`performance`** method computes a performance `metric` for those quantile portfolio returns using any of the measures from the `Metrics` class.

Quantile portfolio returns can be used to:

- Analyze the robustness of alpha factors. Returns which increase monotonically with quantiles are likely to be more reliable that those with positive but non-monotonic relationships.
- Explore the potential interaction between alpha factors with double portfolio sorts in order to discover conditional alpha factors, e.g. a double sort of the trend factor and size factor to analyze how trend factor performance changes by asset size.
- Explore the relationship between time series and cross-sectional factors, and the interaction between the two (i.e. dual strategies). E.g. a time series trend factor conditioned on the cross-sectional trend factor can enhance performance if it leads to a stronger signal. 


## Single Portfolio Sort 

### Time Series Strategy

In [ ]:
port_sort = PortfolioSort(ret_df.tr, 
                   trend_df[[trend_ts_port_perf_table.iloc[0].name]], 
                   factor_bins={trend_ts_port_perf_table.iloc[0].name: ('ts', 3)},
                   fill_na=True
                  )

In [ ]:
trend_ts_quantile_port = port_sort.compute_quantile_portfolios()

In [ ]:
plot_series(trend_ts_quantile_port[trend_ts_port_perf_table.iloc[0].name].unstack().cumsum(),             
            title=f"Factor: {trend_ts_port_perf_table.iloc[0].name}",
            subtitle=f"Time Series Quantiles Portfolio Returns",
            y_label='Cumulative Log Returns')

In [ ]:
trend_ts_quantile_perf = PortfolioSort(ret_df.tr, 
                   trend_df[[trend_ts_port_perf_table.iloc[0].name]], 
                   factor_bins={trend_ts_port_perf_table.iloc[0].name: ('ts', 3)},
                   fill_na=True
                  ).performance()

In [ ]:
plot_bar(trend_ts_quantile_perf,
         axis='horizontal',
         title=f"Factor: {trend_ts_port_perf_table.iloc[0].name}",
         subtitle=f"Time Series Quantiles Portfolio Returns",
         y_label='Quantile',
         x_label='Sharpe ratio'
        )

### Cross-sectional Strategy

In [ ]:
trend_cs_quantile_port = PortfolioSort(ret_df.tr, 
                   trend_df[[trend_cs_port_perf_table.iloc[0].name]], 
                   factor_bins={trend_cs_port_perf_table.iloc[0].name: ('cs', 3)},
                   fill_na=True
                  ).compute_quantile_portfolios()

In [ ]:
plot_series(trend_cs_quantile_port[trend_cs_port_perf_table.iloc[0].name].unstack().cumsum(),             
            title=f"Trend Factor: {trend_cs_port_perf_table.iloc[0].name}",
            subtitle=f"Cross-sectional Quantiles Portfolio Returns",
            y_label='Cumulative Log Returns')

In [ ]:
trend_cs_quantile_perf = PortfolioSort(ret_df.tr, 
                   trend_df[[trend_cs_port_perf_table.iloc[0].name]], 
                   factor_bins={trend_cs_port_perf_table.iloc[0].name: ('cs', 3)},
                   fill_na=True
                  ).performance()

In [ ]:
plot_bar(trend_cs_quantile_perf,
         axis='horizontal',
         title=f"Factor: {trend_cs_port_perf_table.iloc[0].name}",
         subtitle=f"Cross-sectional Quantiles Portfolio Returns",
         y_label='Quantile',
         x_label='Sharpe ratio'
        )

## Double Portfolio Sort

In [ ]:
trend_dual_quantile_port = PortfolioSort(ret_df.tr, 
                   trend_df[['breakout_10' , 'breakout_5']], 
                   factor_bins={'breakout_10': ('ts', 3), 'breakout_5': ('cs', 3)
                               },
                   fill_na=True
                  ).compute_quantile_portfolios()

In [ ]:
plot_series(trend_dual_quantile_port.unstack().cumsum(),             
            title=f"Trend Factor: Breakout",
            subtitle=f"Quantiles Portfolio Returns",
            y_label='Cumulative Log Returns')

In [ ]:
trend_dual_quantile_perf = PortfolioSort(ret_df.tr, 
                   trend_df[['breakout_5' , 'breakout_10']], 
                   factor_bins={'breakout_10': ('ts', 3), 'breakout_5': ('cs', 3)
                               },
                   fill_na=True
                  ).performance(metric='sharpe_ratio')

In [ ]:
plot_heatmap(trend_dual_quantile_perf,
             title=f"Double Portfolio Sort",
             subtitle=f"Quantile Portfolio Returns - Sharpe Ratio",
            )

# Parameter Robustness

## Parameter Grid Search

### Time Series
We explore the parameter space for the best performing time series trend strategy.

### Cross Sectional

We explore the parameter space for the best performing cross-sectional trend strategy.

## Trend Dashboard

In [ ]:
trend_dual_signals_df = Signal(ret_df.tr, trend_df, strategy='dual_ls').compute_dual_signals(signal_type='signal', summary_stat='sum')

In [ ]:
breakout10_signals_bar = trend_dual_signals_df.breakout_10.unstack().iloc[-1].sort_values().dropna()

In [ ]:
plot_bar(pd.concat([breakout10_signals_bar[:10], breakout10_signals_bar[-10:]], axis=1), 
         title="Trend Signals",
         subtitle=f"Breakout 10 day - {trend_df.iloc[-1].name[0].date()}",
         axis='horizontal', 
         y_label='ticker', 
         x_label='signal', 
         add_logo=True,
        )